**NLP LAB 09**

The best dataset to choose is offensive. Indeed, haste is a dataset about comment that disrespect a social or a group of person, but it can be a point of view, when the offensive dataset is about insult someone.

The best would be taking both dataset.

In [9]:

!pip install datasets
!pip install bertopic
!pip install torch
!pip install transformers

In [1]:
from datasets import load_dataset
from bertopic import BERTopic
from transformers import AutoTokenizer, AutoModelForSequenceClassification, RobertaForSequenceClassification
from scipy.special import softmax
from torch import random
import json

2022-11-23 16:54:07.307570: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 16:54:07.859823: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-23 16:54:09.126416: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-23 16:54:09.126608: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
dataset = load_dataset("tweet_eval","offensive")
dataset

Found cached dataset tweet_eval (/home/victor/.cache/huggingface/datasets/tweet_eval/offensive/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11916
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 860
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1324
    })
})

We see that there is not much value in the test dataset  (About 5.7% of all data) and in the validation dataset (About 8.9% of all data)

In [3]:
dataset["train"]["text"][:5]

['@user Bono... who cares. Soon people will understand that they gain nothing from following a phony celebrity. Become a Leader of your people instead or help and support your fellow countrymen.',
 '@user Eight years the republicans denied obama’s picks. Breitbarters outrage is as phony as their fake president.',
 '@user Get him some line help. He is gonna be just fine. As the game went on you could see him progressing more with his reads. He brought what has been missing. The deep ball presence. Now he just needs a little more time',
 '@user @user She is great. Hi Fiona!',
 "@user She has become a parody unto herself? She has certainly taken some heat for being such an....well idiot. Could be optic too  Who know with Liberals  They're all optics.  No substance"]

In [4]:
dataset["train"][90:95]

{'text': ['@user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user @user Thank you for adding me Razor!  Patriots- if you aren’t following my Primary account @user - will you take a minute &amp; Follow me? Thanks &amp; MAGA🇺🇸',
  '@user The lies i keep hearing everyday sigh *INHALES* IM TIRED OF YOU PUTTING YOURSELF DOWN AND IF PPL DONT SEE HOW BEAUTIFUL MY BESTFRIEND IS THEN ITS THEIR PROBLEM You are beautiful ive seen iT so its my place to tell you you are beautiful and i accept nothing less. If not more 😤',
  '@user He is very perceptive Sweetie!',
  '@user Dear Paul McFartney-  You had us at Beatles.  You lost us at President Mad Captain.  Burn in hell-  Jon.""',
  '@user you are also the king of taste'],
 'label': [0, 1, 0, 1, 0]}

We notice that all tag is anonymous. They had been replaced by '@user'.

In [5]:
topic_model_train = BERTopic()
topics_train, probs_train = topic_model_train.fit_transform(dataset["train"]["text"])

topic_model_test = BERTopic()
topics_test, probs_test = topic_model_test.fit_transform(dataset["test"]["text"])

topic_model_validation = BERTopic()
topics_validation, probs_validation = topic_model_validation.fit_transform(dataset["validation"]["text"])


In [6]:


random.manual_seed(42)
topic_model_train.get_topic_info()

,Topic,Count,Name
0,-1,4709,-1_is_you_and_to
1,0,1019,0_gun_control_guns_laws
2,1,368,1_liberals_they_liberal_their
3,2,292,2_antifa_user_like_to
4,3,283,3_right_wrong_user_you
...,...,...,...
119,118,12,118_shit_wtf_eirc_dns
120,119,12,119_terrorist_islamic_removed_another
121,120,11,120_bullshit_insanely_chimed_daffy
122,121,11,121_jimmy_price_offset_mcbuckets


In [7]:
topic_model_test.get_topic_info()

,Topic,Count,Name
0,-1,70,-1_user_the_to_for
1,0,359,0_is_she_you_and
2,1,344,1_the_to_user_of
3,2,37,2_gun_user_control_the
4,3,25,3_you_are_user_sales
5,4,13,4_listen_aclu_black_more
6,5,12,5_florence_hurricane_hurricaneflorence_to


In [8]:
topic_model_validation.get_topic_info()

,Topic,Count,Name
0,-1,6,-1_nfl_american_boycott_to
1,0,1067,0_user_the_is_to
2,1,128,1_user_antifa_the_to
3,2,123,2_gun_control_user_the


We see there is a difference in topic between the validation, test and train dataset. There will be a risk to train model on some topic, but to test it.

In [9]:
task= "offensive"
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)


In [ ]:
model : RobertaForSequenceClassification = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.sa

In [ ]:
file_json = open("tweets.json", 'r')
tweets_json = json.load(file_json)

file_json.close()


In [ ]:


text = "Just go to Hell. You don't deserve it."
encoded_input = tokenizer(tweets_json)
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

scores


In [ ]:
len(dataset["test"]["text"])